In [2]:
import json

def load_vocab_and_merges(vocab_path="vocab.json", merges_path="merges.txt"):
    with open(vocab_path, "r", encoding="utf-8") as f:
        vocab = json.load(f)

    with open(merges_path, "r", encoding="utf-8") as f:
        merges = f.read().splitlines()[1:]  # skip first line which is a header
        merges = [tuple(merge.strip().split()) for merge in merges]

    return vocab, merges


In [3]:
vocab, merges = load_vocab_and_merges()
print("Vocab size:", len(vocab))
print("First 5 merges:", merges[:5])


Vocab size: 50257
First 5 merges: [('Ġ', 't'), ('Ġ', 'a'), ('h', 'e'), ('i', 'n'), ('r', 'e')]


In [4]:
bs = (
    list(range(ord("!"), ord("~") + 1)) +
    list(range(ord("¡"), ord("¬") + 1)) +
    list(range(ord("®"), ord("ÿ") + 1))
)
cs = bs[:]
n = 0
for b in range(2 ** 8):
    if b not in bs:
        bs.append(b)
        cs.append(2 ** 8 + n)
        n += 1
cs = [chr(c) for c in cs]
dict(zip(bs,cs))

{33: '!',
 34: '"',
 35: '#',
 36: '$',
 37: '%',
 38: '&',
 39: "'",
 40: '(',
 41: ')',
 42: '*',
 43: '+',
 44: ',',
 45: '-',
 46: '.',
 47: '/',
 48: '0',
 49: '1',
 50: '2',
 51: '3',
 52: '4',
 53: '5',
 54: '6',
 55: '7',
 56: '8',
 57: '9',
 58: ':',
 59: ';',
 60: '<',
 61: '=',
 62: '>',
 63: '?',
 64: '@',
 65: 'A',
 66: 'B',
 67: 'C',
 68: 'D',
 69: 'E',
 70: 'F',
 71: 'G',
 72: 'H',
 73: 'I',
 74: 'J',
 75: 'K',
 76: 'L',
 77: 'M',
 78: 'N',
 79: 'O',
 80: 'P',
 81: 'Q',
 82: 'R',
 83: 'S',
 84: 'T',
 85: 'U',
 86: 'V',
 87: 'W',
 88: 'X',
 89: 'Y',
 90: 'Z',
 91: '[',
 92: '\\',
 93: ']',
 94: '^',
 95: '_',
 96: '`',
 97: 'a',
 98: 'b',
 99: 'c',
 100: 'd',
 101: 'e',
 102: 'f',
 103: 'g',
 104: 'h',
 105: 'i',
 106: 'j',
 107: 'k',
 108: 'l',
 109: 'm',
 110: 'n',
 111: 'o',
 112: 'p',
 113: 'q',
 114: 'r',
 115: 's',
 116: 't',
 117: 'u',
 118: 'v',
 119: 'w',
 120: 'x',
 121: 'y',
 122: 'z',
 123: '{',
 124: '|',
 125: '}',
 126: '~',
 161: '¡',
 162: '¢',
 163: '£',

In [5]:
def bytes_to_unicode():
    bs = (
        list(range(ord("!"), ord("~") + 1)) +
        list(range(ord("¡"), ord("¬") + 1)) +
        list(range(ord("®"), ord("ÿ") + 1))
    )
    cs = bs[:]
    n = 0
    for b in range(2 ** 8):
        if b not in bs:
            bs.append(b)
            cs.append(2 ** 8 + n)
            n += 1
    cs = [chr(c) for c in cs]
    return dict(zip(bs, cs))


In [6]:
byte_encoder = bytes_to_unicode()

def text_to_tokens(text):
    token_list = [byte_encoder[b] for b in text.encode("utf-8")]
    return token_list


In [7]:
text = "Hello world!"
tokens = text_to_tokens(text)
print(tokens)


['H', 'e', 'l', 'l', 'o', 'Ġ', 'w', 'o', 'r', 'l', 'd', '!']


In [8]:
bpe_ranks = {pair: i for i, pair in enumerate(merges)}


In [9]:
len(vocab), len(merges)

(50257, 50000)

In [10]:
pairs = set()
prev = tokens[0]
for token in tokens[1:]:
    pairs.add((prev, token))
    prev = token
pairs

{('H', 'e'),
 ('d', '!'),
 ('e', 'l'),
 ('l', 'd'),
 ('l', 'l'),
 ('l', 'o'),
 ('o', 'r'),
 ('o', 'Ġ'),
 ('r', 'l'),
 ('w', 'o'),
 ('Ġ', 'w')}

In [11]:
def get_pairs(tokens):
    pairs = set()
    prev = tokens[0]
    for token in tokens[1:]:
        pairs.add((prev, token))
        prev = token
    return pairs


In [12]:
def bpe(token_list, bpe_ranks):
    tokens = token_list.copy()
    while True:
        pairs = get_pairs(tokens)
        if not pairs:
            break

        # Find the highest priority pair to merge (lowest rank)
        candidate_pairs = {pair: bpe_ranks.get(pair, None) for pair in pairs}
        # Filter out pairs not in bpe_ranks
        candidate_pairs = {pair: rank for pair, rank in candidate_pairs.items() if rank is not None}
        
        if not candidate_pairs:
            break
        
        # Pick pair with smallest rank
        best_pair = min(candidate_pairs, key=candidate_pairs.get)
        i = 0
        new_tokens = []
        
        while i < len(tokens):
            # If pair found at position i, merge it
            if i < len(tokens) - 1 and (tokens[i], tokens[i+1]) == best_pair:
                new_tokens.append(tokens[i] + tokens[i+1])
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        
        tokens = new_tokens
    return tokens


In [13]:
text = "Hello world!"
tokens = text_to_tokens(text)

In [14]:
bpe_ranks = {pair: i for i, pair in enumerate(merges)}

text = "Hello world!"
tokens = text_to_tokens(text)
bpe_tokens = bpe(tokens, bpe_ranks)
print(bpe_tokens)


['Hello', 'Ġworld', '!']


In [15]:
def tokens_to_ids(tokens, vocab):
    ids = []
    for token in tokens:
        #if token in vocab:
            ids.append(vocab[token])
        #else:
         #   raise ValueError(f"Token {token} not found in vocab")
    return ids

token_ids = tokens_to_ids(['Hello', 'Ġworld', '!'], vocab)
print(token_ids)


[15496, 995, 0]


In [16]:
import numpy as np

vocab_size = 50257  # GPT-2 vocab size
embed_dim = 768     # GPT-2 embedding dim

# Initialize embedding matrix with small random values
wte = np.random.randn(vocab_size, embed_dim) * 0.02


In [35]:
num_heads=12

In [ ]:
def get_sinusoidal_positional_embeddings(seq_len, d_model):
    position = np.arange(seq_len)
    position=position.reshape(seq_len,1)  # (seq_len, 1)
    div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
    pe = np.zeros((seq_len, d_model))
    pe[:, 0::2] = np.sin(position * div_term)
    pe[:, 1::2] = np.cos(position * div_term)
    return pe

max_seq_len = 512
pos_emb = get_sinusoidal_positional_embeddings(max_seq_len, embed_dim)


In [67]:
wte[token_ids]

array([[-0.00566097, -0.00450344, -0.01896024, ...,  0.00847298,
         0.01568138,  0.00171688],
       [-0.04954254, -0.0120693 ,  0.01837816, ...,  0.00453781,
         0.01500814, -0.02409617],
       [-0.03457359,  0.00445436,  0.00831081, ...,  0.00076492,
        -0.06554725,  0.02365601]])

In [18]:
seq_len = len(token_ids)
input_embeds = wte[token_ids] + pos_emb[:seq_len]


In [19]:
token_ids = [15496, 995, 0]  # example tokens
input_embeds = wte[token_ids] + pos_emb[:len(token_ids)]
print(input_embeds.shape)  # (seq_len, embed_dim)


(3, 768)


In [20]:
def init_attention_weights(embed_dim, num_heads):
    head_dim = embed_dim // num_heads
    W_q = np.random.randn(embed_dim, embed_dim) * 0.02
    W_k = np.random.randn(embed_dim, embed_dim) * 0.02
    W_v = np.random.randn(embed_dim, embed_dim) * 0.02
    W_o = np.random.randn(embed_dim, embed_dim) * 0.02
    return W_q, W_k, W_v, W_o


In [21]:
def split_heads(x, num_heads):
    # x: (seq_len, embed_dim)
    seq_len, embed_dim = x.shape
    head_dim = embed_dim // num_heads
    x = x.reshape(seq_len, num_heads, head_dim)
    x = x.transpose(1, 0, 2)  # (num_heads, seq_len, head_dim)
    return x


In [22]:
def causal_attention(Q, K, V):
    # Q, K, V: (num_heads, seq_len, head_dim)
    d_k = Q.shape[-1]
    scores = np.matmul(Q, K.transpose(0, 2, 1)) / np.sqrt(d_k)  # (num_heads, seq_len, seq_len)

    # Causal mask
    seq_len = Q.shape[1]
    mask = np.triu(np.ones((seq_len, seq_len)), k=1).astype(bool)
    mask = np.expand_dims(mask, axis=0)  # (1, seq_len, seq_len) to broadcast over heads
    scores = np.where(mask, -np.inf, scores)

    # Use stable softmax
    max_scores = np.max(scores, axis=-1, keepdims=True)
    scores = scores - max_scores  # prevent overflow
    exp_scores = np.exp(scores)
    attn_weights = exp_scores / np.sum(exp_scores, axis=-1, keepdims=True) + 1e-8

    output = np.matmul(attn_weights, V)  # (num_heads, seq_len, head_dim)
    return output


In [23]:
def multi_head_attention(x, W_q, W_k, W_v, W_o, num_heads):
    # x: (seq_len, embed_dim)
    Q = x @ W_q  # (seq_len, embed_dim)
    K = x @ W_k
    V = x @ W_v

    Q = split_heads(Q, num_heads)  # (num_heads, seq_len, head_dim)
    K = split_heads(K, num_heads)
    V = split_heads(V, num_heads)

    attn_output = causal_attention(Q, K, V)  # (num_heads, seq_len, head_dim)

    # Concatenate heads
    attn_output = attn_output.transpose(1, 0, 2).reshape(x.shape)  # (seq_len, embed_dim)

    # Final projection
    output = attn_output @ W_o  # (seq_len, embed_dim)
    return output


In [24]:
# Step 1: Init weights
W_q, W_k, W_v, W_o = init_attention_weights(embed_dim, num_heads=12)

# Step 2: Run attention
attn_out = multi_head_attention(input_embeds, W_q, W_k, W_v, W_o, num_heads=12)
print(attn_out.shape)  # Should be (seq_len, embed_dim)


(3, 768)


In [25]:
def init_mlp_weights(embed_dim, hidden_dim):
    W1 = np.random.randn(embed_dim, hidden_dim) * 0.02
    b1 = np.zeros((hidden_dim,))
    W2 = np.random.randn(hidden_dim, embed_dim) * 0.02
    b2 = np.zeros((embed_dim,))
    return W1, b1, W2, b2

def mlp(x, W1, b1, W2, b2):
    x = x @ W1 + b1  # (seq_len, hidden_dim)
    x = np.maximum(0, x)  # ReLU
    x = x @ W2 + b2  # (seq_len, embed_dim)
    return x


In [26]:
hidden_dim = 4 * embed_dim  # GPT-2 uses 4x hidden size for MLP
W1, b1, W2, b2 = init_mlp_weights(embed_dim, hidden_dim)

mlp_out = mlp(attn_out, W1, b1, W2, b2)
print(mlp_out.shape)  # should be (3, 768)


(3, 768)


In [28]:
# --- Step 1: Output projection (tie weights with embedding matrix) ---
# wte: (vocab_size, embed_dim)
# input_embeds: (seq_len, embed_dim)
# output_logits: (seq_len, vocab_size)
output_logits = input_embeds @ wte.T

# --- Step 2: Convert logits to predicted token IDs ---
# For each position, pick the most probable token
pred_token_ids = np.argmax(output_logits, axis=-1)  # shape: (seq_len,)
print("Predicted token IDs:", pred_token_ids)

# --- Step 3: Decode predicted token IDs to string ---
def decode_tokens(token_ids, vocab):
    id_to_token = {v: k for k, v in vocab.items()}
    tokens = [id_to_token.get(token_id, "<unk>") for token_id in token_ids]
    return "".join(tokens).replace("Ġ", " ")  # replace GPT-2 space symbol

# Decode to text
decoded_text = decode_tokens(pred_token_ids, vocab)
print("Decoded text:", decoded_text)


Predicted token IDs: [19386 19386 19386]
Decoded text:  integrate integrate integrate


In [30]:
def encode_text(text, merges, vocab, bpe_ranks):
    def get_pairs(word):
        pairs = set()
        prev_char = word[0]
        for char in word[1:]:
            pairs.add((prev_char, char))
            prev_char = char
        return pairs

    def bpe_token(word, bpe_ranks):
        word = tuple(word)
        pairs = get_pairs(word)
        while pairs:
            pair = min(pairs, key=lambda p: bpe_ranks.get(p, float('inf')))
            if pair not in bpe_ranks:
                break
            first, second = pair
            new_word = []
            i = 0
            while i < len(word):
                if i < len(word) - 1 and word[i] == first and word[i + 1] == second:
                    new_word.append(first + second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            word = tuple(new_word)
            pairs = get_pairs(word)
        return word

    def text_to_tokens(text):
        # Add space before each word to simulate GPT-2 BPE behavior
        return ["Ġ" + word if i > 0 else word for i, word in enumerate(text.strip().split())]

    tokens = text_to_tokens(text)
    bpe_tokens = []
    for token in tokens:
        word_pieces = bpe_token(token, bpe_ranks)
        bpe_tokens.extend(word_pieces)

    token_ids = [vocab.get(t, vocab.get("<unk>", 0)) for t in bpe_tokens]
    return token_ids


In [33]:
def layer_norm(x, eps=1e-5):
    mean = np.mean(x, axis=-1, keepdims=True)
    var = np.var(x, axis=-1, keepdims=True)
    norm_x = (x - mean) / np.sqrt(var + eps)
    return norm_x


In [36]:
# x: input to attention
# 1. Pre-Norm before Attention
x_norm = layer_norm(input_embeds)
attn_out = multi_head_attention(x_norm, W_q, W_k, W_v, W_o, num_heads)
x = input_embeds + attn_out  # Residual

# 2. Pre-Norm before MLP
x_norm2 = layer_norm(x)
mlp_out = mlp(x_norm2, W1, b1, W2, b2)
x = x + mlp_out  # Residual

# 3. Final layer norm before logits (optional)
x = layer_norm(x)

# Project to logits
logits = x @ wte.T
